In [ ]:
import pandas as pd
input_file = 'measuring_hate_speech.csv'
df=pd.read_csv(input_file)

<ipython-input-1-4d35d9e97a66>:3: DtypeWarning: Columns (80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,118,119,120,121,122,123,124,125,126,127,128,129,130) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(input_file)


In [ ]:
# Calculate 'emotionaldistavg'
df['emotionaldistavg'] = df.apply(
    lambda row: row[['respect', 'sentiment', 'insult', 'dehumanize', 'humiliate', 'status']].mean()
    if row['hate_speech_score'] < 6.5 else None, axis=1)

# Calculate 'provokingviolenceavg'
df['provokingviolenceavg'] = df[['violence', 'genocide', 'dehumanize', 'attack_defend']].mean(axis=1)

# Function to calculate 'provokingviolence'
def calculate_provoking_violence(row):
    avg = row['provokingviolenceavg']
    if 0 <= avg < 1.35:
        return 0
    elif 1.35 <= avg < 1.5:
        return 1
    elif 1.5 <= avg < 3.16:
        return 2
    elif avg >= 3.16:
        return 3
    return None

# Function to calculate 'emotionaldistress'
def calculate_emotional_distress(row):
    avg = row['emotionaldistavg']
    if row['hate_speech_score'] > 6.5:
        return 3
    elif avg is not None:
        if 0 <= avg < 1.35:
            return 0
        elif 1.35 <= avg < 3.1:
            return 1
        elif 3.1 <= avg <= 4:
            return 2
    return None

# Function to adjust 'provokingviolence'
def adjust_provoking_violence(row):
    if row['provokingviolence'] == 2:
        if row['violence'] == 0 and row['genocide'] == 0 and (row['dehumanize'] in [2, 3] or row['attack_defend'] in [2, 3]):
            return 1
        elif row['violence'] == 0 and row['genocide'] == 0 and (row['dehumanize'] in [0, 1] or row['attack_defend'] in [0, 1]):
            return 0
    return row['provokingviolence']

# Apply calculations and adjustments
df['emotionaldistress'] = df.apply(calculate_emotional_distress, axis=1)
df['provokingviolence'] = df.apply(calculate_provoking_violence, axis=1)
df['provokingviolence'] = df.apply(adjust_provoking_violence, axis=1)

# Calculate 'individualharassmentavg'
df['individualharassmentavg'] = df[['status', 'insult', 'attack_defend', 'dehumanize', 'humiliate']].mean(axis=1)

# Function to calculate 'individualharrassment'
def calculate_individual_harassment(row):
    avg = row['individualharassmentavg']
    if 0 <= avg <= 1.1:
        return 0
    elif 1.1 < avg < 2.8:
        return 1
    elif 2.8 <= avg < 3.75:
        return 2
    elif 3.75 <= avg <= 4:
        return 3
    return None

# Apply individual harassment calculation
df['individualharrassment'] = df.apply(calculate_individual_harassment, axis=1)

# Function to calculate 'hatespeechintensity'
def calculate_hatespeech_intensity(row):
    if row['hatespeech'] in [0, 1]:
        return row['hatespeech']
    elif row['hatespeech'] == 2:
        if (row[['sentiment', 'respect', 'insult', 'dehumanize', 'humiliate', 'violence', 'attack_defend']].mean()) >= 3.8:
            return 3
        else:
            return 2
    return None

# Apply the function to generate 'hatespeechintensity'
df['hatespeechintensity'] = df.apply(calculate_hatespeech_intensity, axis=1)

# Select columns for the first output file
columns_to_keep = [
    'comment_id', 'annotator_id', 'platform', 'sentiment', 'respect', 'insult', 'humiliate', 'status',
    'dehumanize', 'violence', 'genocide', 'attack_defend', 'hatespeech', 'text', 'hate_speech_score',
    'emotionaldistavg', 'emotionaldistress', 'provokingviolenceavg', 'provokingviolence',
    'individualharassmentavg', 'individualharrassment', 'hatespeechintensity'
]

# Save the preprocessed dataset into 'updated_dataset_preprocessed.csv'
new_df = df[columns_to_keep]
new_df.to_csv('updated_dataset_preprocessed.csv', index=False)

# Load the new dataframe and select columns for the second output file
new_df = pd.read_csv('updated_dataset_preprocessed.csv')
columns_to_save = ['comment_id', 'text', 'emotionaldistress', 'provokingviolence', 'individualharrassment', 'hatespeechintensity']

# Save the second version into 'dataset_v2.csv'
new_df_v2 = new_df[columns_to_save]
new_df_v2.to_csv('dataset_v2.csv', index=False)

# Print sample data from both datasets
print("Updated Dataset (Preprocessed):")
print(new_df.head())

print("\nDataset V2:")
print(new_df_v2.head())

# Print the length of the new dataset
print(f"\nLength of Dataset V2: {len(new_df_v2)}")


Updated Dataset (Preprocessed):
   comment_id  annotator_id  platform  sentiment  respect  insult  humiliate  \
0       47777         10873         3          0        0       0          0   
1       39773          2790         2          0        0       0          0   
2       47101          3379         3          4        4       4          4   
3       43625          7365         3          2        3       2          1   
4       12538           488         0          4        4       4          4   

   status  dehumanize  violence  ...  hatespeech  \
0       2           0         0  ...           0   
1       2           0         0  ...           0   
2       4           4         0  ...           2   
3       2           0         0  ...           0   
4       4           4         4  ...           2   

                                                text  hate_speech_score  \
0  Yes indeed. She sort of reminds me of the elde...              -3.90   
1  The trans women readi

In [ ]:
new_df1 = pd.read_csv('updated_dataset_preprocessed.csv')

new_columns = [
    'emotionaldistavg', 'emotionaldistress', 'provokingviolenceavg', 'provokingviolence',
    'individualharassmentavg', 'individualharrassment', 'hatespeechintensity'
]

nan_counts = new_df1[new_columns].isna().sum()

print("Number of NaN values in each new column:")
print(nan_counts)

Number of NaN values in each new column:
emotionaldistavg           0
emotionaldistress          0
provokingviolenceavg       0
provokingviolence          0
individualharassmentavg    0
individualharrassment      0
hatespeechintensity        0
dtype: int64


In [ ]:
import pandas as pd
import re
import torch
from transformers import BertTokenizer, BertModel
from bs4 import BeautifulSoup
import string

df = pd.read_csv('updated_dataset_preprocessed.csv')


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model.to(device)

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r'@\w+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

df['cleaned_text'] = df['text'].apply(clean_text)

def tokenize_text(text):

    encoded = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    return encoded

def embed_text(text):
    tokens = tokenize_text(text)
    input_ids = tokens['input_ids'].to(device)
    attention_mask = tokens['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
    return cls_embedding


df['embedded_text'] = df['cleaned_text'].apply(embed_text)

df['embedded_text'] = df['embedded_text'].apply(lambda x: x.tolist())

columns_to_save = [
    'comment_id', 'embedded_text', 'emotionaldistress',
    'provokingviolence', 'individualharrassment', 'hatespeechintensity'
]

processed_df = df[columns_to_save]

processed_df.to_csv('processed_bert_embedded_dataset.csv', index=False)

print(processed_df.head())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Using device: cuda


<ipython-input-4-4183d4a84cbd>:23: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


   comment_id                                      embedded_text  \
0       47777  [-0.08673834055662155, 0.022910749539732933, -...   
1       39773  [0.14396293461322784, -0.007813084870576859, 0...   
2       47101  [-0.2818196713924408, 0.31229910254478455, -0....   
3       43625  [-0.014131826348602772, 0.1035425141453743, -0...   
4       12538  [-0.4389498233795166, -0.44695088267326355, -0...   

   emotionaldistress  provokingviolence  individualharrassment  \
0                  0                  0                      0   
1                  0                  0                      0   
2                  2                  2                      3   
3                  1                  0                      1   
4                  2                  2                      3   

   hatespeechintensity  
0                    0  
1                    0  
2                    2  
3                    0  
4                    3  


In [ ]:
import pandas as pd

file_path = 'processed_bert_embedded_dataset.csv'
df = pd.read_csv(file_path)
df_filtered = df[df['hatespeechintensity'] != 0]
df_final = df_filtered.drop('hatespeechintensity', axis=1)
df_final.to_csv('final_embedded_dataset.csv', index=False)
print(df_final.head())


   comment_id                                      embedded_text  \
2       47101  [-0.2818196713924408, 0.31229910254478455, -0....   
4       12538  [-0.4389498233795166, -0.44695088267326355, -0...   
6       13168  [0.07826884090900421, 0.41834592819213867, 0.2...   
7       17034  [0.21192587912082672, 0.45486536622047424, -0....   
9        1006  [-0.16936618089675903, 0.056792594492435455, 0...   

   emotionaldistress  provokingviolence  individualharrassment  
2                  2                  2                      3  
4                  2                  2                      3  
6                  2                  1                      2  
7                  2                  0                      2  
9                  2                  2                      3  


In [ ]:
import pandas as pd

df = pd.read_csv('final_embedded_dataset.csv')
multilabel_df = df[['comment_id', 'embedded_text', 'emotionaldistress', 'provokingviolence', 'individualharrassment']]
multilabel_df.to_csv('multilabel.csv', index=False)
print(multilabel_df.head())

   comment_id                                      embedded_text  \
0       47101  [-0.2818196713924408, 0.31229910254478455, -0....   
1       12538  [-0.4389498233795166, -0.44695088267326355, -0...   
2       13168  [0.07826884090900421, 0.41834592819213867, 0.2...   
3       17034  [0.21192587912082672, 0.45486536622047424, -0....   
4        1006  [-0.16936618089675903, 0.056792594492435455, 0...   

   emotionaldistress  provokingviolence  individualharrassment  
0                  2                  2                      3  
1                  2                  2                      3  
2                  2                  1                      2  
3                  2                  0                      2  
4                  2                  2                      3  


In [ ]:
# Count occurrences of 0, 1, 2, and 3 (or 0, 1, 2) in specified columns

def count_occurrences(df, columns):
    for col in columns:
        counts = df[col].value_counts()
        print(f"Counts for column '{col}':")
        print(counts)
        print("-" * 20)

columns_to_check = ['emotionaldistress', 'provokingviolence', 'individualharrassment']
count_occurrences(df, columns_to_check)

Counts for column 'emotionaldistress':
emotionaldistress
2    5408
1    3731
0      86
Name: count, dtype: int64
--------------------
Counts for column 'provokingviolence':
provokingviolence
2    5203
0    2368
1     980
3     674
Name: count, dtype: int64
--------------------
Counts for column 'individualharrassment':
individualharrassment
2    4736
1    2875
3    1548
0      66
Name: count, dtype: int64
--------------------


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, RepeatVector, TimeDistributed, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from collections import Counter
from sklearn.metrics import accuracy_score

In [ ]:
# Load your dataset
data = pd.read_csv('multilabel.csv')

# Extract features and labels
X_embedded = np.array(data['embedded_text'].apply(lambda x: np.fromstring(x[1:-1], sep=',')).tolist())  # Convert embedded_text to array
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values  # Labels


In [ ]:
# Convert y to binary format (multi-label)
y_binary = (y > 0).astype(int)  # Presence or absence


In [ ]:
# Split the data
X_train, X_val, y_train, y_val = train_test_split(X_embedded, y_binary, test_size=0.2, random_state=42)


In [ ]:
# Function to calculate inverse frequency weights for each class
def compute_inverse_freq_weights(y):
    class_weights = {}
    for i in range(y.shape[1]):  # Loop over each label
        label_counts = Counter(y[:, i])  # Count occurrences of 0 and 1 in each label
        total = len(y)  # Total number of samples
        # Calculate weight for each class as inverse of its frequency
        class_weights[i] = {
            0: total / (2 * label_counts[0]),  # Weight for class 0 (negative)
            1: total / (2 * label_counts[1])   # Weight for class 1 (positive)
        }
    return class_weights

# Compute inverse frequency weights for the training labels
class_weights = compute_inverse_freq_weights(y_binary)


In [ ]:
# Reshape the data for CNN input
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)


In [ ]:
# Convert the class weights into a tensor for use in the loss function
output_dim = len(class_weights)
class_weights_tensor = np.array([[class_weights[i][0], class_weights[i][1]] for i in range(output_dim)], dtype=np.float32)
class_weights_tensor = tf.constant(class_weights_tensor)


# Define custom weighted binary crossentropy loss function
def weighted_binary_crossentropy(y_true, y_pred):
    # Calculate the binary crossentropy loss
    bce_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)

    # Reshape bce_loss to add a dimension for the output classes
    bce_loss = tf.expand_dims(bce_loss, axis=-1) # Add dimension to match weights

    # Compute the weights for the true labels (0 or 1)
    # using broadcasting for efficient computation
    weights = tf.where(tf.equal(y_true, 1),
                       class_weights_tensor[..., 1],  # Weight for class 1 (positive)
                       class_weights_tensor[..., 0])  # Weight for class 0 (negative)


    # Apply the weights to the binary crossentropy loss
    weighted_loss = bce_loss * weights

    # Return the mean weighted loss across samples and time steps
    return tf.reduce_mean(weighted_loss)



In [ ]:
# Reshape y for the decoder to match the output
num_time_steps = X_train.shape[1]  # This should match the sequence length in your data
y_train_seq = np.repeat(y_train[:, np.newaxis, :], num_time_steps, axis=1)
y_val_seq = np.repeat(y_val[:, np.newaxis, :], num_time_steps, axis=1)

# Ensure the types are float32
y_train_seq = y_train_seq.astype(np.float32)
y_val_seq = y_val_seq.astype(np.float32)


In [ ]:
# Model creation
def create_seq2seq_cnn_model(input_shape, output_dim):
    # Encoder
    encoder_inputs = Input(shape=input_shape)
    x = Conv1D(filters=32, kernel_size=2, activation='relu', padding='same')(encoder_inputs)
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(filters=64, kernel_size=2, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)

    # Flatten the output before Dense
    x = Flatten()(x)
    encoder_outputs = Dense(128, activation='relu')(x)

    # Expand dimensions to match the expected input for LSTM
    encoder_outputs = RepeatVector(num_time_steps)(encoder_outputs)

    # Decoder
    decoder_lstm = LSTM(32, return_sequences=True)(encoder_outputs)
    decoder_outputs = TimeDistributed(Dense(output_dim, activation='sigmoid'))(decoder_lstm)

    # Create the model
    model = Model(encoder_inputs, decoder_outputs)
    return model

# Initialize and compile the model
input_shape = (X_embedded.shape[1], 1)  # Input shape
output_dim = y_binary.shape[1]  # Output shape
model = create_seq2seq_cnn_model(input_shape, output_dim)
model.compile(optimizer='adam', loss=weighted_binary_crossentropy, metrics=['accuracy'])


In [ ]:
# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train_seq,
                    batch_size=32,
                    epochs=10,
                    validation_data=(X_val, y_val_seq),
                    callbacks=[early_stopping])


Epoch 1/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 57s 206ms/step - accuracy: 0.0034 - loss: 0.6319 - val_accuracy: 0.4257 - val_loss: 0.7541
Epoch 2/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 9s 113ms/step - accuracy: 0.6133 - loss: 0.6215 - val_accuracy: 0.9968 - val_loss: 0.7348
Epoch 3/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 9s 101ms/step - accuracy: 0.9676 - loss: 0.5278 - val_accuracy: 0.9907 - val_loss: 0.7294
Epoch 4/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 10s 103ms/step - accuracy: 0.1615 - loss: 0.4963 - val_accuracy: 0.9968 - val_loss: 0.7224
Epoch 5/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 11s 115ms/step - accuracy: 0.8004 - loss: 0.7005 - val_accuracy: 0.9968 - val_loss: 0.8256
Epoch 6/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 10s 118ms/step - accuracy: 0.8271 - loss: 0.6435 - val_accuracy: 0.0352 - val_loss: 0.7110
Epoch 7/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 8s 100ms/step - accuracy: 0.1966 - loss: 0.6121 - val_accuracy: 0.9025 - val_loss: 0.7550
Epoch 8/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 11s 112ms/step - accuracy: 0.7470 - loss: 0.5539 - val_accurac

In [ ]:

# Predictions on the validation set
y_pred_probs = model.predict(X_val)

# Set a threshold for binary classification based on probabilities
threshold = 0.5
y_pred = (y_pred_probs[:, -1, :] > threshold).astype(int)  # Get predictions from the last time step




20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 388ms/step


In [ ]:
# Print classification report for each label
print("Classification Report:")
print(classification_report(y_val, y_pred, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Calculate overall accuracy by comparing the exact match of all labels
overall_accuracy = accuracy_score(y_val, y_pred)

# Print the overall accuracy
print(f"\nOverall Accuracy: {overall_accuracy:.4f}")

Classification Report:
                       precision    recall  f1-score   support

   Emotional Distress       0.99      1.00      1.00       623
   Provoking Violence       0.78      0.54      0.64       460
Individual Harassment       0.99      1.00      1.00       623

            micro avg       0.95      0.88      0.91      1706
            macro avg       0.92      0.85      0.88      1706
         weighted avg       0.93      0.88      0.90      1706
          samples avg       0.95      0.88      0.90      1706


Overall Accuracy: 0.5374


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Display sample examples from the validation set
def display_examples(X, y_true, y_pred, num_examples=25):
    for i in range(num_examples):
        print(f"Example {i+1}:")
        print(f"True Labels (y): {y_true[i]}")
        print(f"Predicted Labels: {y_pred[i]}\n")

# Display sample examples from the validation set
display_examples(X_val, y_val, y_pred)


Example 1:
True Labels (y): [1 0 1]
Predicted Labels: [1 1 1]

Example 2:
True Labels (y): [1 0 1]
Predicted Labels: [1 0 1]

Example 3:
True Labels (y): [1 0 1]
Predicted Labels: [1 0 1]

Example 4:
True Labels (y): [1 1 1]
Predicted Labels: [1 0 1]

Example 5:
True Labels (y): [1 1 1]
Predicted Labels: [1 0 1]

Example 6:
True Labels (y): [1 1 1]
Predicted Labels: [1 0 1]

Example 7:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 8:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 9:
True Labels (y): [1 1 1]
Predicted Labels: [1 0 1]

Example 10:
True Labels (y): [1 1 1]
Predicted Labels: [1 0 1]

Example 11:
True Labels (y): [1 0 0]
Predicted Labels: [1 0 1]

Example 12:
True Labels (y): [1 1 1]
Predicted Labels: [1 0 1]

Example 13:
True Labels (y): [0 0 0]
Predicted Labels: [1 0 1]

Example 14:
True Labels (y): [1 1 1]
Predicted Labels: [1 0 1]

Example 15:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 16:
True Labels (y): [1 1 1]
Predicted La

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from transformers import AdamW

# Load the dataset
data = pd.read_csv('multilabel.csv')

# Convert the 'embedded_text' to numpy arrays
data['embedded_text'] = data['embedded_text'].apply(lambda x: np.fromstring(x[1:-1], sep=','))
X = np.array(data['embedded_text'].tolist())  # Convert to numpy array
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values

# Convert y to binary format (multi-label)
y_binary = (y > 0).astype(int)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Create a custom Dataset class
class MultilabelDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.embeddings[idx], dtype=torch.float32),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float32)
        }

# Create DataLoaders
train_dataset = MultilabelDataset(X_train, y_train)
val_dataset = MultilabelDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Define a transformer-based model for multilabel classification
class TransformerMultilabelClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(TransformerMultilabelClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)  # Increase input dimension for better learning
        self.dropout = nn.Dropout(0.3)  # Dropout layer to prevent overfitting
        self.fc2 = nn.Linear(256, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout
        x = torch.sigmoid(self.fc2(x))  # Sigmoid for multilabel output
        return x

# Instantiate the model
input_dim = X.shape[1]  # Number of features from BERT embeddings
output_dim = y_binary.shape[1]  # Number of labels
model = TransformerMultilabelClassifier(input_dim, output_dim)

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss for multilabel
optimizer = AdamW(model.parameters(), lr=0.001)

# Training function
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Train the model
train_model(model, train_loader, criterion, optimizer)

# Evaluation function
def evaluate_model(model, val_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids)
            predictions.append(outputs.cpu().numpy())
            true_labels.append(labels.cpu().numpy())

    return np.vstack(predictions), np.vstack(true_labels)

# Evaluate the model
y_pred, y_true = evaluate_model(model, val_loader)

# Evaluate the model
y_pred, y_true = evaluate_model(model, val_loader)

# Binarize predictions
y_pred_binary = (y_pred > 0.5).astype(int)

# Print classification report
print(classification_report(y_true, y_pred_binary, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Calculate overall accuracy
overall_accuracy = accuracy_score(y_true, y_pred_binary)
print(f"Overall Accuracy: {overall_accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Loss: 0.2317
Epoch 2/10, Loss: 0.2091
Epoch 3/10, Loss: 0.2009
Epoch 4/10, Loss: 0.1997
Epoch 5/10, Loss: 0.1928
Epoch 6/10, Loss: 0.1898
Epoch 7/10, Loss: 0.1851
Epoch 8/10, Loss: 0.1833
Epoch 9/10, Loss: 0.1793
Epoch 10/10, Loss: 0.1787
                       precision    recall  f1-score   support

   Emotional Distress       0.99      1.00      1.00       623
   Provoking Violence       0.75      0.95      0.84       460
Individual Harassment       0.99      1.00      1.00       623

            micro avg       0.91      0.99      0.95      1706
            macro avg       0.91      0.98      0.94      1706
         weighted avg       0.92      0.99      0.95      1706
          samples avg       0.91      0.98      0.94      1706

Overall Accuracy: 0.7281


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import torch

# Set the device to GPU if available, otherwise fallback to CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the correct device
model.to(device)

# Define a function to display examples from the validation set
def display_examples(X, y_true, num_examples=25):
    # Ensure that y_pred is in binary format
    y_pred = (X > 0.4).astype(int)  # Apply threshold to get binary predictions

    for i in range(num_examples):
        print(f"Example {i + 1}:")
        print(f"True Labels (y): {y_true[i]}")
        print(f"Predicted Labels: {y_pred[i]}\n")

# Prepare the input tensor and move it to the correct device
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)

# Set model to evaluation mode and get predictions
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    y_pred = model(X_val_tensor)  # Get model predictions

# Convert predictions to binary format
y_pred_binary = (y_pred.cpu().numpy() > 0.5).astype(int)  # Move predictions back to CPU and threshold

# Now, display the examples
display_examples(y_pred_binary, y_val, num_examples=25)



Example 1:
True Labels (y): [1 0 1]
Predicted Labels: [1 0 1]

Example 2:
True Labels (y): [1 0 1]
Predicted Labels: [1 1 1]

Example 3:
True Labels (y): [1 0 1]
Predicted Labels: [1 1 1]

Example 4:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 5:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 6:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 7:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 8:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 9:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 10:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 11:
True Labels (y): [1 0 0]
Predicted Labels: [1 1 1]

Example 12:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 13:
True Labels (y): [0 0 0]
Predicted Labels: [1 1 1]

Example 14:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 15:
True Labels (y): [1 1 1]
Predicted Labels: [1 1 1]

Example 16:
True Labels (y): [1 1 1]
Predicted La

In [43]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Load the dataset
data = pd.read_csv('final_embedded_dataset.csv')

# Convert 'embedded_text' to numpy arrays
data['embedded_text'] = data['embedded_text'].apply(lambda x: np.fromstring(x[1:-1], sep=','))
X = np.array(data['embedded_text'].tolist())
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values

# Convert y to binary format (multi-label)
y_binary = (y > 0).astype(int)

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Adjusted Hybrid CNN-Attention model without LSTM
class HybridMultilabelClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(HybridMultilabelClassifier, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=128, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool1d(2)

        # Fully connected layer after flattening
        self.fc1 = nn.Linear(128 * (input_dim // 2), 64)

        # Attention mechanism
        self.attention = nn.Linear(64, 1)

        # Final fully connected layer for classification
        self.fc2 = nn.Linear(64, output_dim)

    def forward(self, x):
        print("Input shape:", x.shape)
        x = x.unsqueeze(1)  # Add channel dimension for Conv1d, Shape: [batch_size, 1, input_dim]
        print("After unsqueeze for Conv1d:", x.shape)

        x = torch.relu(self.conv1(x))  # Shape: [batch_size, 128, input_dim]
        print("After Conv1d:", x.shape)

        x = self.pool1(x)  # Shape: [batch_size, 128, input_dim/2]
        print("After MaxPool1d:", x.shape)

        x = x.view(x.size(0), -1)  # Flatten for fully connected layer, Shape: [batch_size, 128 * (input_dim // 2)]
        print("After flattening:", x.shape)

        x = torch.relu(self.fc1(x))  # Shape: [batch_size, 64]
        print("After first fully connected layer:", x.shape)

        attention_weights = torch.softmax(self.attention(x), dim=1)  # Shape: [batch_size, 1]
        x = attention_weights * x  # Apply attention
        print("After attention mechanism:", x.shape)

        x = torch.sigmoid(self.fc2(x))  # Sigmoid for multilabel output
        print("After final fully connected layer:", x.shape)
        return x



# Instantiate model, loss function, optimizer
input_dim = X.shape[1]  # Number of features in embeddings
output_dim = y_binary.shape[1]  # Number of labels
# Instantiate and train this modified model
model = HybridMultilabelClassifier(input_dim=X.shape[1], output_dim=y_binary.shape[1]).to(device)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss for multilabel
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

# Training function
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

# Train the model
train_model(model, train_loader, criterion, optimizer)

# Evaluation function
def evaluate_model(model, val_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids)
            predictions.append(outputs.cpu().numpy())
            true_labels.append(labels.cpu().numpy())

    return np.vstack(predictions), np.vstack(true_labels)

# Evaluate the model
y_pred, y_true = evaluate_model(model, val_loader)

# Binarize predictions
y_pred_binary = (y_pred > 0.5).astype(int)

# Print classification report
print(classification_report(y_true, y_pred_binary, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Calculate overall accuracy
overall_accuracy = accuracy_score(y_true, y_pred_binary)
print(f"Overall Accuracy: {overall_accuracy:.4f}")


NameError: name 'train_loader' is not defined

In [ ]:
from sklearn.metrics import precision_recall_curve

# Calculate optimal thresholds based on precision-recall trade-off
def calculate_optimal_thresholds(y_true, y_pred_prob):
    thresholds = []
    for i in range(y_true.shape[1]):
        # Calculate precision-recall curve for the label
        precision, recall, threshold = precision_recall_curve(y_true[:, i], y_pred_prob[:, i])

        # Get threshold that maximizes F1 (balanced precision and recall)
        f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
        best_threshold = threshold[np.argmax(f1_scores)]
        thresholds.append(best_threshold)
    return thresholds

# Calculate predictions with label-specific thresholds
def predict_with_custom_thresholds(model, val_loader, thresholds):
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].cpu().numpy()

            outputs = model(input_ids).cpu().numpy()
            predictions.append(outputs)
            true_labels.append(labels)

    predictions = np.vstack(predictions)
    true_labels = np.vstack(true_labels)

    # Apply label-specific thresholds
    pred_labels = np.zeros_like(predictions)
    for i, threshold in enumerate(thresholds):
        pred_labels[:, i] = (predictions[:, i] > threshold).astype(int)

    return pred_labels, true_labels

# Get prediction probabilities on validation set
y_pred_prob, y_true = evaluate_model(model, val_loader)

# Calculate optimal thresholds per label
optimal_thresholds = calculate_optimal_thresholds(y_true, y_pred_prob)
print("Optimal thresholds for each label:", optimal_thresholds)

# Make predictions with these thresholds
y_pred_binary, y_true_binary = predict_with_custom_thresholds(model, val_loader, optimal_thresholds)

# Display classification report with custom thresholds
print(classification_report(y_true_binary, y_pred_binary, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))



Input shape: torch.Size([16, 768])
After unsqueeze for Conv1d: torch.Size([16, 1, 768])
After Conv1d: torch.Size([16, 128, 768])
After MaxPool1d: torch.Size([16, 128, 384])
After flattening: torch.Size([16, 49152])
After first fully connected layer: torch.Size([16, 64])
After attention mechanism: torch.Size([16, 64])
After final fully connected layer: torch.Size([16, 3])
Input shape: torch.Size([16, 768])
After unsqueeze for Conv1d: torch.Size([16, 1, 768])
After Conv1d: torch.Size([16, 128, 768])
After MaxPool1d: torch.Size([16, 128, 384])
After flattening: torch.Size([16, 49152])
After first fully connected layer: torch.Size([16, 64])
After attention mechanism: torch.Size([16, 64])
After final fully connected layer: torch.Size([16, 3])
Input shape: torch.Size([16, 768])
After unsqueeze for Conv1d: torch.Size([16, 1, 768])
After Conv1d: torch.Size([16, 128, 768])
After MaxPool1d: torch.Size([16, 128, 384])
After flattening: torch.Size([16, 49152])
After first fully connected layer: to

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Updated function to make predictions on a few samples with label-specific thresholds
def predict_samples_with_thresholds(model, val_loader, thresholds, num_samples=25):
    model.eval()  # Set model to evaluation mode
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].cpu().numpy()  # True labels for comparison

            # Get model predictions
            outputs = model(input_ids).cpu().numpy()  # Move predictions to CPU
            predictions.append(outputs)
            true_labels.append(labels)

            if len(predictions) >= num_samples:
                break  # Stop after the desired number of samples

    # Convert predictions and true labels to numpy arrays
    predictions = np.vstack(predictions)
    true_labels = np.vstack(true_labels)

    # Apply label-specific thresholds to predictions
    pred_labels = np.zeros_like(predictions)
    for i, threshold in enumerate(thresholds):
        pred_labels[:, i] = (predictions[:, i] > threshold).astype(int)

    # Display true and predicted labels for the samples
    for i in range(num_samples):
        print(f"Sample {i + 1}")
        print(f"True Labels: {true_labels[i]}")
        print(f"Predicted Labels: {pred_labels[i]}\n")

# Now use the function to predict and display samples
predict_samples_with_thresholds(model, val_loader, optimal_thresholds, num_samples=25)


Input shape: torch.Size([16, 768])
After unsqueeze for Conv1d: torch.Size([16, 1, 768])
After Conv1d: torch.Size([16, 128, 768])
After MaxPool1d: torch.Size([16, 128, 384])
After flattening: torch.Size([16, 49152])
After first fully connected layer: torch.Size([16, 64])
After attention mechanism: torch.Size([16, 64])
After final fully connected layer: torch.Size([16, 3])
Input shape: torch.Size([16, 768])
After unsqueeze for Conv1d: torch.Size([16, 1, 768])
After Conv1d: torch.Size([16, 128, 768])
After MaxPool1d: torch.Size([16, 128, 384])
After flattening: torch.Size([16, 49152])
After first fully connected layer: torch.Size([16, 64])
After attention mechanism: torch.Size([16, 64])
After final fully connected layer: torch.Size([16, 3])
Input shape: torch.Size([16, 768])
After unsqueeze for Conv1d: torch.Size([16, 1, 768])
After Conv1d: torch.Size([16, 128, 768])
After MaxPool1d: torch.Size([16, 128, 384])
After flattening: torch.Size([16, 49152])
After first fully connected layer: to

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

# Load the dataset
data = pd.read_csv('multilabel.csv')

# Convert the 'embedding' column to numpy arrays
data['embedded_text'] = data['embedded_text'].apply(lambda x: np.fromstring(x[1:-1], sep=','))
X = np.array(data['embedded_text'].tolist())  # Convert to numpy array
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values

# Convert y to binary format (multi-label)
y_binary = (y > 0).astype(int)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Define an XGBoost DMatrix for GPU
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

# Set up XGBoost parameters for GPU usage
params = {
    'objective': 'binary:logistic',  # Binary classification for each label
    'tree_method': 'gpu_hist',       # Use GPU acceleration
    'eval_metric': 'logloss',        # Log loss for binary classification
    'predictor': 'gpu_predictor',
    'learning_rate': 0.01,
    'max_depth': 6,
    'verbosity': 1
}

# Train one model per label (multi-label setup)
models = []
for i in range(y_train.shape[1]):
    print(f"Training model for label {i+1}/{y_train.shape[1]}")
    # Train the model for each label
    model = xgb.train(params, dtrain, num_boost_round=100)
    models.append(model)

# Predict and evaluate
y_pred = np.zeros(y_val.shape)
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

# Load the dataset
data = pd.read_csv('multilabel.csv')

# Convert the 'embedding' column to numpy arrays
data['embedded_text'] = data['embedded_text'].apply(lambda x: np.fromstring(x[1:-1], sep=','))
X = np.array(data['embedded_text'].tolist())  # Convert to numpy array
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values

# Convert y to binary format (multi-label)
y_binary = (y > 0).astype(int)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Define an XGBoost DMatrix for GPU
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

# Set up XGBoost parameters for GPU usage
params = {
    'objective': 'binary:logistic',  # Binary classification for each label
    'tree_method': 'gpu_hist',       # Use GPU acceleration
    'eval_metric': 'logloss',        # Log loss for binary classification
    'predictor': 'gpu_predictor',
    'learning_rate': 0.01,
    'max_depth': 6,
    'verbosity': 1
}

# Train one model per label (multi-label setup)
models = []
for i in range(y_train.shape[1]):
    print(f"Training model for label {i+1}/{y_train.shape[1]}")
    # Train the model for each label
    model = xgb.train(params, dtrain, num_boost_round=100)
    models.append(model)

# Predict and evaluate
y_pred = np.zeros(y_val.shape)
for i, model in enumerate(models):
    # Predict probabilities for the current label only
    y_pred[:, i] = model.predict(dval)[:, i]

# Binarize predictions
y_pred_binary = (y_pred > 0.5).astype(int)

# Print classification report for each label
print("Classification Report:")
print(classification_report(y_val, y_pred_binary, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Calculate overall accuracy
overall_accuracy = accuracy_score(y_val, y_pred_binary)
print(f"Overall Accuracy: {overall_accuracy:.4f}")

# Define a function to display examples from the validation set
def display_examples(X, y_true, num_examples=10):
    for i in range(num_examples):
        print(f"Example {i + 1}:")
        print(f"True Labels: {y_true[i]}")
        print(f"Predicted Labels: {X[i]}\n")

# Display examples from the validation set
display_examples(y_pred_binary, y_val, num_examples=10)


# Binarize predictions
y_pred_binary = (y_pred > 0.5).astype(int)

# Print classification report for each label
print("Classification Report:")
print(classification_report(y_val, y_pred_binary, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Calculate overall accuracy
overall_accuracy = accuracy_score(y_val, y_pred_binary)
print(f"Overall Accuracy: {overall_accuracy:.4f}")

# Define a function to display examples from the validation set
def display_examples(X, y_true, num_examples=10):
    for i in range(num_examples):
        print(f"Example {i + 1}:")
        print(f"True Labels: {y_true[i]}")
        print(f"Predicted Labels: {X[i]}\n")

# Display examples from the validation set
display_examples(y_pred_binary, y_val, num_examples=10)

Training model for label 1/3


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:34:30] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:34:30] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Training model for label 2/3


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:34:33] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:34:33] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Training model for label 3/3


...

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:34:45] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:34:45] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Training model for label 3/3


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:34:48] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:34:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Classification Report:
                       precision    recall  f1-score   support

   Emotional Distress       0.99      1.00      1.00       623
   Provoking Violence       0.73      1.00      0.84       460
Individual Harassment       0.99      1.00      1.00       623

            micro avg       0.90      1.00      0.95      1706
            macro avg       0.90      1.00      0.95      1706
         weighted avg       0.92      1.00      0.95      1706
          samples avg       0.90      0.99      0.94      1706

Overall Accuracy: 0.7313
Example 1:
True Labels: [1 0 1]
Predicted Labels: [1 1 1]

Example 2:
True Labels: [1 0 1]
Predicted Labels: [1 1 1]

Example 3:
True Labels: [1 0 1]
Predicted Labels: [1 1 1]

Example 4:
True Labels: [1 1 1]
Predicted Labels: [1 1 1]

Example 5:
True Labels: [1 1 1]
Predicted Labels: [1 1 1]

Example 6:
True Labels: [1 1 1]
Predicted Labels: [1 1 1]

Example 7:
True Labels: [1 1 1]
Predicted Labels: [1 1 1]

Example 8:
True Labels: [1 1 1]


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:34:52] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [2]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Load the dataset
data = pd.read_csv('final_embedded_dataset.csv')

# Convert 'embedded_text' to numpy arrays
data['embedded_text'] = data['embedded_text'].apply(lambda x: np.fromstring(x[1:-1], sep=','))
X = np.array(data['embedded_text'].tolist())
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values

# Convert y to binary format (multi-label)
y_binary = (y > 0).astype(int)

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Custom Dataset class
class MultilabelDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.embeddings[idx], dtype=torch.float32),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float32)
        }

# Create DataLoaders
train_dataset = MultilabelDataset(X_train, y_train)
val_dataset = MultilabelDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Hybrid Model with CNN, BiLSTM, and Separate Attention Layers
class HybridCNNBiLSTMClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(HybridCNNBiLSTMClassifier, self).__init__()

        # CNN layers
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(2)

        # BiLSTM layer
        self.bilstm = nn.LSTM(input_size=256, hidden_size=128, num_layers=1, batch_first=True, bidirectional=True)

        # Shared fully connected layer
        self.fc_shared = nn.Linear(128 * 2 * (input_dim // 2), 128)  # Adjusted for BiLSTM output

        # Separate attention and classification layers for each label
        self.attention_emotional = nn.Linear(128, 1)
        self.fc_emotional = nn.Linear(128, 1)

        self.attention_violence = nn.Linear(128, 1)
        self.fc_violence = nn.Linear(128, 1)

        self.attention_harassment = nn.Linear(128, 1)
        self.fc_harassment = nn.Linear(128, 1)

    def forward(self, x):
        # CNN
        x = x.unsqueeze(1)  # Add channel dimension for Conv1d
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.pool(x)  # Shape: [batch_size, 256, input_dim / 2]

        # BiLSTM
        x = x.permute(0, 2, 1)  # Reshape for LSTM: [batch_size, seq_len, features]
        lstm_out, _ = self.bilstm(x)  # Shape: [batch_size, input_dim / 2, 256]

        # Flatten for fully connected layer
        x = lstm_out.contiguous().view(lstm_out.size(0), -1)
        shared_features = torch.relu(self.fc_shared(x))  # Shape: [batch_size, 128]

        # Emotional Distress Classification with Attention
        attn_weights_emotional = torch.softmax(self.attention_emotional(shared_features), dim=1)
        emotional_features = attn_weights_emotional * shared_features
        emotional_output = torch.sigmoid(self.fc_emotional(emotional_features))

        # Provoking Violence Classification with Attention
        attn_weights_violence = torch.softmax(self.attention_violence(shared_features), dim=1)
        violence_features = attn_weights_violence * shared_features
        violence_output = torch.sigmoid(self.fc_violence(violence_features))

        # Individual Harassment Classification with Attention
        attn_weights_harassment = torch.softmax(self.attention_harassment(shared_features), dim=1)
        harassment_features = attn_weights_harassment * shared_features
        harassment_output = torch.sigmoid(self.fc_harassment(harassment_features))

        # Concatenate outputs for multilabel classification
        return torch.cat((emotional_output, violence_output, harassment_output), dim=1)

# Instantiate model, loss function, optimizer
input_dim = X.shape[1]  # Number of features in embeddings
output_dim = y_binary.shape[1]  # Number of labels

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HybridCNNBiLSTMClassifier(input_dim=input_dim, output_dim=output_dim).to(device)

# Loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

# Training function
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

# Train the model
train_model(model, train_loader, criterion, optimizer)

# Evaluation function
def evaluate_model(model, val_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].cpu().numpy()

            outputs = model(input_ids).cpu().numpy()
            predictions.append(outputs)
            true_labels.append(labels)

    return np.vstack(predictions), np.vstack(true_labels)

# Evaluate the model
y_pred, y_true = evaluate_model(model, val_loader)

# Binarize predictions
y_pred_binary = (y_pred > 0.5).astype(int)

# Print classification report
print(classification_report(y_true, y_pred_binary, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Calculate overall accuracy
overall_accuracy = accuracy_score(y_true, y_pred_binary)
print(f"Overall Accuracy: {overall_accuracy:.4f}")


Epoch 1/10, Loss: 0.1638
Epoch 2/10, Loss: 0.1523
Epoch 3/10, Loss: 0.1484
Epoch 4/10, Loss: 0.1450
Epoch 5/10, Loss: 0.1400
Epoch 6/10, Loss: 0.1349
Epoch 7/10, Loss: 0.1277
Epoch 8/10, Loss: 0.1199
Epoch 9/10, Loss: 0.1126
Epoch 10/10, Loss: 0.1062
                       precision    recall  f1-score   support

   Emotional Distress       0.99      1.00      0.99     10887
   Provoking Violence       0.85      0.93      0.89      9012
Individual Harassment       0.99      1.00      1.00     10906

            micro avg       0.95      0.98      0.96     30805
            macro avg       0.94      0.97      0.96     30805
         weighted avg       0.95      0.98      0.96     30805
          samples avg       0.95      0.97      0.95     30805

Overall Accuracy: 0.8001


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Load and preprocess the dataset
data = pd.read_csv('final_embedded_dataset.csv')
data['embedded_text'] = data['embedded_text'].apply(lambda x: np.fromstring(x[1:-1], sep=','))
X = np.array(data['embedded_text'].tolist())  # Features
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values
y_binary = (y > 0).astype(int)  # Binarize labels for multi-label classification

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # Add sequence dimension
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).unsqueeze(1)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MultiHeadAttn(nn.Module):
    def __init__(self):
        super().__init__()
        self.multi_attn = nn.MultiheadAttention(embed_dim=768, num_heads=12, batch_first=True)

    def forward(self, q, k, v):
        mv_1, _ = self.multi_attn(q, k, v)
        return mv_1

class MultiLabelClassificationNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_labels):
        super(MultiLabelClassificationNet, self).__init__()
        self.multiAttn1 = MultiHeadAttn()
        self.multiAttn2 = MultiHeadAttn()
        self.multiAttn3 = MultiHeadAttn()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_labels)

    def forward(self, x):
        # Apply attention layers
        attn_out1 = self.multiAttn1(x, x, x)
        attn_out2 = self.multiAttn2(attn_out1, attn_out1, attn_out1)
        attn_out3 = self.multiAttn3(attn_out2, attn_out2, attn_out2)

        # Use mean pooling to reduce sequence dimension for fully connected layer
        x = attn_out3.mean(dim=1)  # (batch_size, embed_dim)

        # Pass through fully connected layers
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))  # Sigmoid activation for multi-label classification
        return x

# Hyperparameters
input_dim = 768  # Example input dimension from embeddings
hidden_dim = 128
num_labels = y_train.shape[1]
batch_size = 32
num_epochs = 10
learning_rate = 0.001

# Model, loss, and optimizer
model = MultiLabelClassificationNet(input_dim=input_dim, hidden_dim=hidden_dim, num_labels=num_labels).to(device)
criterion = nn.BCELoss()  # Binary cross-entropy loss for multi-label
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Move tensors to device
X_train_tensor, y_train_tensor = X_train_tensor.to(device), y_train_tensor.to(device)
X_val_tensor, y_val_tensor = X_val_tensor.to(device), y_val_tensor.to(device)

# Training loop
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluation
model.eval()
with torch.no_grad():
    y_val_pred = model(X_val_tensor)
    y_val_pred_binary = (y_val_pred > 0.5).int()

# Convert predictions and labels to numpy for evaluation
y_val_pred_np = y_val_pred_binary.cpu().numpy()
y_val_np = y_val_tensor.cpu().numpy()

# Print classification report
print("Classification Report:")
print(classification_report(y_val_np, y_val_pred_np, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Calculate overall accuracy
overall_accuracy = accuracy_score(y_val_np, y_val_pred_np)
print(f"Overall Accuracy: {overall_accuracy:.4f}")

# Define a function to display examples from the validation set
def display_examples(X, y_true, y_pred, num_examples=10):
    for i in range(num_examples):
        print(f"Example {i + 1}:")
        print(f"True Labels: {y_true[i]}")
        print(f"Predicted Labels: {y_pred[i]}\n")

# Display examples from the validation set
display_examples(X_val, y_val_np, y_val_pred_np, num_examples=10)


Epoch [1/10], Loss: 0.6940
Epoch [2/10], Loss: 0.6004
Epoch [3/10], Loss: 0.1962
Epoch [4/10], Loss: 0.3432
Epoch [5/10], Loss: 0.2317
Epoch [6/10], Loss: 0.3752
Epoch [7/10], Loss: 0.3084
Epoch [8/10], Loss: 0.3561
Epoch [9/10], Loss: 0.2153
Epoch [10/10], Loss: 0.2124
Classification Report:
                       precision    recall  f1-score   support

   Emotional Distress       0.99      1.00      1.00     10887
   Provoking Violence       0.82      0.99      0.90      9012
Individual Harassment       0.99      1.00      1.00     10906

            micro avg       0.93      1.00      0.97     30805
            macro avg       0.93      1.00      0.96     30805
         weighted avg       0.94      1.00      0.97     30805
          samples avg       0.94      0.99      0.96     30805

Overall Accuracy: 0.8169
Example 1:
True Labels: [1. 1. 1.]
Predicted Labels: [1 1 1]

Example 2:
True Labels: [1. 1. 1.]
Predicted Labels: [1 1 1]

Example 3:
True Labels: [1. 1. 1.]
Predicted Label

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [3]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.6 MB/s eta 0:00:00


In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv


In [22]:
# Load and preprocess the dataset
data = pd.read_csv('final_embedded_dataset.csv')
data['embedded_text'] = data['embedded_text'].apply(lambda x: np.fromstring(x[1:-1], sep=','))
X = np.array(data['embedded_text'].tolist())  # Features
y = data[['emotionaldistress', 'provokingviolence', 'individualharrassment']].values
y_binary = (y > 0).astype(int)  # Binarize labels for multi-label classification

In [23]:
# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y_binary, test_size=0.2, random_state=42)

In [24]:
# Construct graph using k-NN
knn = NearestNeighbors(n_neighbors=10, metric='cosine')  # k-NN with cosine distance
knn.fit(X_train)
distances, indices = knn.kneighbors(X_train)


In [25]:
# Construct edge_index for graph (using k-NN results)
edge_index = []
for i in range(len(indices)):
    for j in indices[i]:
        edge_index.append([i, j])
edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

In [26]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

In [27]:
# Create a graph data object for PyTorch Geometric
train_data = Data(x=X_train_tensor, edge_index=edge_index, y=y_train_tensor)

In [28]:
# GCN model
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_labels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_labels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return torch.sigmoid(x)  # Multi-label classification

In [29]:
# Hyperparameters
input_dim = X_train.shape[1]  # Feature dimension (size of embedded vector)
hidden_dim = 128
num_labels = y_train.shape[1]
num_epochs = 10
learning_rate = 0.001

In [30]:
# Model, loss, and optimizer
model = GCN(input_dim=input_dim, hidden_dim=hidden_dim, num_labels=num_labels)
criterion = nn.BCELoss()  # Binary cross-entropy loss for multi-label
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [31]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    output = model(train_data)

    # Calculate loss
    loss = criterion(output, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/10], Loss: 0.5480
Epoch [2/10], Loss: 0.3526
Epoch [3/10], Loss: 0.2948
Epoch [4/10], Loss: 0.2697
Epoch [5/10], Loss: 0.2537
Epoch [6/10], Loss: 0.2413
Epoch [7/10], Loss: 0.2294
Epoch [8/10], Loss: 0.2127
Epoch [9/10], Loss: 0.2203
Epoch [10/10], Loss: 0.2246


In [38]:
# Construct graph for validation data using k-NN
knn_val = NearestNeighbors(n_neighbors=10, metric='cosine')
knn_val.fit(X_val)
distances_val, indices_val = knn_val.kneighbors(X_val)

In [39]:
# Construct edge_index for validation data (using k-NN results)
edge_index_val = []
for i in range(len(indices_val)):
    for j in indices_val[i]:
        edge_index_val.append([i, j])
edge_index_val = torch.tensor(edge_index_val, dtype=torch.long).t().contiguous()

In [40]:
# Create validation graph data object
val_data = Data(x=X_val_tensor, edge_index=edge_index_val, y=y_val_tensor)

In [41]:
# Evaluation
model.eval()
with torch.no_grad():
    output = model(val_data)  # Use the validation data
    y_pred_binary_val = (output > 0.5).int()

# Ensure correct shape for the validation set
y_pred_binary_val_np = y_pred_binary_val.cpu().numpy()
y_val_np = y_val_tensor.cpu().numpy()

In [42]:

# Classification Report
print("Classification Report:")
print(classification_report(y_val_np, y_pred_binary_val_np, target_names=['Emotional Distress', 'Provoking Violence', 'Individual Harassment']))

# Overall accuracy
overall_accuracy = accuracy_score(y_val_np, y_pred_binary_val_np)
print(f"Overall Accuracy: {overall_accuracy:.4f}")

Classification Report:
                       precision    recall  f1-score   support

   Emotional Distress       0.99      1.00      1.00     10887
   Provoking Violence       0.83      0.97      0.89      9012
Individual Harassment       0.99      1.00      1.00     10906

            micro avg       0.94      0.99      0.96     30805
            macro avg       0.94      0.99      0.96     30805
         weighted avg       0.94      0.99      0.97     30805
          samples avg       0.94      0.99      0.96     30805

Overall Accuracy: 0.8105


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
